In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
from ta.volume import VolumeWeightedAveragePrice

In [3]:
bnb = pd.read_csv('bnb.csv')
bnb.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1523956260,0,7.0,12.4195,12.4195,12.4101,12.4195,794.70,12.411386,-0.004366
1,1523956320,0,33.0,12.4195,12.4195,12.4001,12.4150,1117.73,12.407532,-0.003940
2,1523956380,0,32.0,12.4150,12.4195,12.4003,12.4100,1062.37,12.401607,-0.003153
3,1523956440,0,38.0,12.4006,12.4100,12.3931,12.4000,2259.55,12.399251,-0.003429
4,1523956500,0,79.0,12.4000,12.4210,12.3930,12.3942,7113.37,12.406144,-0.002187


In [4]:
bnb.iloc[889,0].astype('datetime64[s]')

numpy.datetime64('2018-04-18T00:00:00')

In [5]:
bnb.iloc[2329,0].astype('datetime64[s]')
# 1440 diff

numpy.datetime64('2018-04-19T00:00:00')

In [6]:
# most accurate way to reproduce return so far
def log_return(series, period):
    return np.exp(-np.log(series).diff(periods=-period).shift(-1)) - 1

In [7]:
log_return(bnb['Close'], 15).head(20)

0    -0.004366
1    -0.003940
2    -0.003153
3    -0.003429
4    -0.002187
5    -0.003029
6    -0.003223
7    -0.005509
8    -0.001832
9    -0.002989
10   -0.005512
11   -0.003640
12   -0.003228
13   -0.003102
14   -0.003452
15   -0.002492
16   -0.002524
17   -0.001885
18   -0.000980
19   -0.002207
Name: Close, dtype: float64

In [8]:
start = bnb.iloc[889,0].astype('datetime64[s]') # 2018-04-17T09:11:00
end = bnb.iloc[-2,0].astype('datetime64[s]') # 2021-09-20T23:59:00

In [11]:
# convert minute data to daily
open_price = [bnb.loc[i, 'Open'] for i in range(889, len(bnb)-1, 1440)]
close_price = [bnb.loc[i, 'Close'] for i in range(2328, len(bnb)-1, 1440)]
timestamp = [bnb.loc[i, 'timestamp'] for i in range(889, len(bnb)-1, 1440)]
low = [bnb.iloc[i:i+1440]['Low'].min() for i in range(889, len(bnb)-1, 1440)]
high = [bnb.iloc[i:i+1440]['High'].max() for i in range(889, len(bnb)-1, 1440)]
count = [bnb.iloc[i:i+1440]['Count'].sum() for i in range(889, len(bnb)-1, 1440)]
target = [bnb.iloc[i:i+1440]['Target'].sum() for i in range(889, len(bnb)-1, 1440)]
volume = [bnb.iloc[i:i+1440]['Volume'].sum() for i in range(889, len(bnb)-1, 1440)]

In [12]:
# new df
df = pd.DataFrame({'timestamp':timestamp, 'Count':count, 'Open':open_price, 'High':high, 'Low':low, 
                   'Close':close_price, 'Volume':volume, 'Target':target})

In [13]:
# generate VWAP
vwap = VolumeWeightedAveragePrice(df['High'], df['Low'], df['Close'], df['Volume'])
df['VWAP'] = vwap.volume_weighted_average_price()

In [40]:
df

,timestamp,Count,Open,High,Low,Close,Volume,Target,VWAP
0,2018-04-18,50890.0,11.7770,12.4500,11.511,12.2999,2.739482e+06,0.519802,NaN
1,2018-04-19,42568.0,12.2999,12.4400,12.020,12.3610,2.401089e+06,0.166809,NaN
2,2018-04-20,54544.0,12.3502,12.7241,11.850,12.5858,2.737433e+06,-0.197935,NaN
3,2018-04-21,97777.0,12.5740,13.7400,12.380,13.3302,4.556037e+06,1.055788,NaN
4,2018-04-22,49904.0,13.3689,13.3987,12.800,12.8888,2.564431e+06,-0.526323,NaN
...,...,...,...,...,...,...,...,...,...
1247,2021-09-16,441885.0,431.2535,435.0000,415.600,424.2605,1.084272e+06,-0.008861,433.453897
1248,2021-09-17,410140.0,424.2285,426.0000,400.340,406.5745,8.910700e+05,-0.306504,429.462282
1249,2021-09-18,385184.0,406.8265,420.0000,401.500,410.6780,7.768988e+05,-0.007885,424.063897
1250,2021-09-19,392540.0,410.7365,419.0000,402.800,408.4295,7.756844e+05,0.219586,419.878409


In [45]:
tickers = ['GC=F', 'SI=F', 'CL=F', 'EURUSD=X', '^TNX', 'SPY', 'DIA', 'QQQ', 'IWM', '^VIX']
sup = pdr.get_data_yahoo(tickers, start, end)
sup = sup['Close']
sup.reset_index(inplace=True, drop=False)
df['timestamp'] = df['timestamp'].astype('datetime64[s]')
sup['timestamp'] = sup['Date'].astype('datetime64[s]')

In [46]:
dat = df.merge(sup, on='timestamp')

In [47]:
dat.isna().sum()

timestamp      0
Count          0
Open           0
High           0
Low            0
Close          0
Volume         0
Target         0
VWAP          11
Date           0
GC=F         140
SI=F         140
CL=F         140
EURUSD=X     110
^TNX         140
SPY          140
DIA          140
QQQ          140
IWM          140
^VIX         140
dtype: int64